In [180]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import requests
import json
import csv
import os
from datetime import datetime 

In [181]:
import os
files = os.listdir("../../data/")
files.sort()
files

['BNBBKRW', 'BTCBKRW', 'ETHBKRW']

In [182]:
def get_figure_for_df(df, name):
   return go.Candlestick(
          name= name,
          x=df['open time'],
          open=df['open'],
          high=df['high'],
          low=df['low'],
          close=df['close']
      )

def get_file(file):
    df = pd.read_csv(f'../../data/{file}')
    df['open time'] = pd.to_datetime(df['open time'], unit='ms')
#     std = df['high'].std()
#     mean = df['high'].mean()
#     df['open'] = (df['open'] - mean) / std
#     df['high'] = (df['high'] - mean) / std
#     df['low'] = (df['low'] - mean) / std
    return df

In [183]:
def candlestick(files):
  first = get_file(files[0])
  max_ = first['high'].max()
  min_ = first['low'].min()
  event_time = datetime.strptime("2020-03-30T16:57:41.000Z", '%Y-%m-%dT%H:%M:%S.%fZ') #first.loc[250, 'open time']

  figures = []
  for file in files:
      df = get_file(file)
      max_ = max(max_, df['high'].max())
      min_ = min(min_, df['low'].min())
      print(df.loc[250])
      figure = get_figure_for_df(df, file)
      figures.append(figure)

  fig = go.Figure(data=figures)
  fig.update_layout(
    title= {
     'text': ':D',
     'y':0.9,
     'x':0.5,
     'xanchor': 'center',
     'yanchor': 'top'
    },
   font=dict(
    family="Courier New, monospace",
    size=20,
    color="#7f7f7f"
  ))
  fig.add_shape(
    # Line Vertical
    dict(
        type="line",
        x0=event_time,
        y0=min_,
        x1=event_time,
        y1=max_,
        line=dict(
            color="RoyalBlue",
            width=3
        )
  ))
  fig.show()
 
 
candlestick(files[1:4])

open time                       2020-04-06 05:10:00
open                                    8.45816e+06
high                                    8.45816e+06
low                                     8.45816e+06
close                                   8.45816e+06
volume                                            0
close time                            1586149859999
quote asset volume                                0
number of trades                                  0
taker buy base asset volume                       0
taker buy quote asset volume                      0
ignore                                            0
Name: 250, dtype: object
open time                       2020-04-06 05:12:00
open                                         181081
high                                         181081
low                                          181081
close                                        181081
volume                                            0
close time                            1

In [179]:
df = pd.read_csv('../../data/ADAUSDT')
df['open time'] = pd.to_datetime(df['open time'], unit='ms')
df.head()
df['high'].max()


FileNotFoundError: [Errno 2] File b'../../data/ADAUSDT' does not exist: b'../../data/ADAUSDT'

In [55]:
pd.to_datetime(df['open time'], unit='ms')

0     2020-01-15 21:56:00
1     2020-01-15 21:57:00
2     2020-01-15 21:58:00
3     2020-01-15 21:59:00
4     2020-01-15 22:00:00
              ...        
995   2020-01-16 14:31:00
996   2020-01-16 14:32:00
997   2020-01-16 14:33:00
998   2020-01-16 14:34:00
999   2020-01-16 14:35:00
Name: open time, Length: 1000, dtype: datetime64[ns]